In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#from pyspark import SparkFiles
#spark.sparkContext.addFile('https://raw.githubusercontent.com/erwindrarusli/machine-learning-linear-regression/master/Ecommerce%20Customers')
#df = spark.read.options(inferSchema='True', header='True').csv(SparkFiles.get('Ecommerce Customers'))
df = spark.read.format('libsvm').load('/content/drive/MyDrive/Colab Notebooks/sample_libsvm_data.txt')

In [ ]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
df.describe().toPandas()

,summary,label
0,count,100
1,mean,0.57
2,stddev,0.49756985195624287
3,min,0.0
4,max,1.0


In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(df)
trainingSummary = lrModel.summary

In [ ]:
trainingSummary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126974...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170280...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200599...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695573...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102630...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [ ]:
predictionAndLabels = lrModel.evaluate(df)
predictionAndLabels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126974...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170280...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200599...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695573...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102630...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [ ]:
predictionAndLabels = predictionAndLabels.predictions.select('label', 'prediction')

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
acc = evaluator.evaluate(predictionAndLabels)


In [ ]:
acc

1.0